<a href="https://colab.research.google.com/github/suyash091/covid-chestxray-dataset/blob/master/Main_COVID_X_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone 'https://github.com/suyash091/covid-chestxray-dataset'
!git clone https://github.com/qubvel/efficientnet.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 814 (delta 1), reused 0 (delta 0), pack-reused 769
Receiving objects: 100% (814/814), 152.40 MiB | 42.53 MiB/s, done.
Resolving deltas: 100% (239/239), done.
Checking out files: 100% (376/376), done.
Cloning into 'efficientnet'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 292 (delta 4), reused 5 (delta 1), pack-reused 280
Receiving objects: 100% (292/292), 835.16 KiB | 7.80 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [0]:
from numpy.random import seed
import random
seed(42)
import os
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

import efficientnet.efficientnet.keras as efn
import pandas as pd

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
import keras
from keras.optimizers import SGD, adam, Adadelta

In [0]:
def target(string):
  if string=='COVID-19':
    return string
  elif string=='No Finding':
    return 'Nothing'
  else:
    return "Others"

In [0]:
df=pd.read_csv('/content/covid-chestxray-dataset/metadata.csv')
df= df[['filename','finding']]
df['finding']=df['finding'].apply(target)
dfother=pd.read_csv('/content/covid-chestxray-dataset/Others.csv')
dfother.index +=len(df)
dfnothing=pd.read_csv('/content/covid-chestxray-dataset/Nothing.csv')
dfnothing.index +=int(len(df)+len(dfother))
dfbal=pd.read_csv('/content/covid-chestxray-dataset/added.csv')
dfbal.index +=int(len(df)+len(dfother)+len(dfnothing))

In [0]:
frames = [df, dfother, dfnothing, dfbal]

df = pd.concat(frames)
df= df[['filename','finding']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
df.describe()

,filename,finding
count,365,365
unique,364,3
top,person1003_virus_1685.jpeg,Others
freq,2,123


In [0]:
len(df)

365

In [0]:
df.finding.unique()

array(['COVID-19', 'Others', 'Nothing'], dtype=object)

In [0]:
len(df[df['finding']=='COVID-19'])-len(df[df['finding']=='Others'])

-2

In [0]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.1)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory='/content/covid-chestxray-dataset/images',
        x_col="filename",
        y_col='finding',
        target_size=(1024, 1024),
        batch_size=4,
        subset = 'training' ,
        class_mode='categorical')

validation_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory='/content/covid-chestxray-dataset/images',
        x_col="filename",
        y_col='finding',
        target_size=(1024, 1024),
        batch_size=4,
        subset="validation",
        class_mode='categorical')

Found 328 validated image filenames belonging to 3 classes.
Found 36 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [0]:
efficient_net = efn.EfficientNetB7(
    weights='imagenet',
    input_shape=(1024,1024,3),
    include_top=False
)
efficient_net.trainable = False













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
258441216/258434480 [==============================] - 4s 0us/step


In [0]:
##CNN + FC 8192 x 2 + GAP + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(units = 8192))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 8192))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence


In [0]:
#sequence = DataSequence(dataframe, batch_size)
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcI.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.8537 val == 0.6389

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5000
82/82 [==============================] - 191s 2s/step - loss: 1.0196 - acc: 0.4970 - val_loss: 1.0075 - val_acc: 0.5556
Epoch 2/5000
82/82 [==============================] - 150s 2s/step - loss: 0.9358 - acc: 0.5884 - val_loss: 1.0017 - val_acc: 0.3333
Epoch 3/5000
82/82 [==============================] - 151s 2s/step - loss: 0.8750 - acc: 0.6494 - val_loss: 1.0403 - val_acc: 0.3333
Epoch 4/5000
82/82 [==============================] - 150s 2s/step - loss: 0.7801 - acc: 0.7317 - val_loss: 1.0267 - val_acc: 0.2222
Epoch 5/5000
82/82 [==============================] - 151s 2s/step - loss: 0.7597 - acc: 0.6921 - val_loss: 0.9881 - val_acc: 0.4167
Epoch 6/5000
82/82 [==============================] - 151s 2s/step - loss: 0.7213 - acc: 0.7409 - val_loss: 0.9975 - val_acc: 0.3611
Epoch 7/5000
82/82 [==============================] - 150s 2s/step - loss: 0.7027 - acc: 0.7317 - val_loss

Process ForkPoolWorker-104:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process ForkPoolWorker-103:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File 

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 8192 x 1 + GAP + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 8192))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence


In [0]:
#sequence = DataSequence(dataframe, batch_size)
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.8232 val == 0.7222

Epoch 1/5000
82/82 [==============================] - 179s 2s/step - loss: 1.0530 - acc: 0.4543 - val_loss: 0.9578 - val_acc: 0.6667
Epoch 2/5000
82/82 [==============================] - 144s 2s/step - loss: 0.9820 - acc: 0.5884 - val_loss: 0.9321 - val_acc: 0.6389
Epoch 3/5000
82/82 [==============================] - 144s 2s/step - loss: 0.9478 - acc: 0.6220 - val_loss: 0.8760 - val_acc: 0.7222
Epoch 4/5000
82/82 [==============================] - 144s 2s/step - loss: 0.8782 - acc: 0.6860 - val_loss: 0.8479 - val_acc: 0.6667
Epoch 5/5000
82/82 [==============================] - 144s 2s/step - loss: 0.8499 - acc: 0.6951 - val_loss: 0.8537 - val_acc: 0.6389
Epoch 6/5000
82/82 [==============================] - 144s 2s/step - loss: 0.8042 - acc: 0.7043 - val_loss: 0.8573 - val_acc: 0.6944
Epoch 7/5000
82/82 [==============================] - 144s 2s/step - loss: 0.7705 - acc: 0.7378 - val_loss: 0.8241 - val_acc: 0.6389
Epoch 8/5000
82/82 [==============================] - 144s 2s/step - 

Process ForkPoolWorker-204:
Process ForkPoolWorker-203:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/iterator.py", line 65, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/iterator.py", line 238, in _get_batches_of_transformed_samples
    x

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 4092 x 1 + GAP + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 4096))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.7896 val == 0.5556

Epoch 1/5000
82/82 [==============================] - 182s 2s/step - loss: 1.0624 - acc: 0.4299 - val_loss: 0.9935 - val_acc: 0.4444
Epoch 2/5000
82/82 [==============================] - 143s 2s/step - loss: 0.9884 - acc: 0.5091 - val_loss: 0.9746 - val_acc: 0.3333
Epoch 3/5000
82/82 [==============================] - 143s 2s/step - loss: 0.9481 - acc: 0.6189 - val_loss: 0.9743 - val_acc: 0.3611
Epoch 4/5000
82/82 [==============================] - 142s 2s/step - loss: 0.9338 - acc: 0.6738 - val_loss: 0.9527 - val_acc: 0.3333
Epoch 5/5000
82/82 [==============================] - 143s 2s/step - loss: 0.8792 - acc: 0.6646 - val_loss: 0.9194 - val_acc: 0.4444
Epoch 6/5000
82/82 [==============================] - 144s 2s/step - loss: 0.8419 - acc: 0.6707 - val_loss: 0.8973 - val_acc: 0.5278
Epoch 7/5000
82/82 [==============================] - 143s 2s/step - loss: 0.8040 - acc: 0.7012 - val_loss: 0.8871 - val_acc: 0.4722
Epoch 8/5000
82/82 [==============================] - 143s 2s/step - 

Process ForkPoolWorker-310:
Process ForkPoolWorker-311:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
 

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 1024 x 1 + GAP + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 1024))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.6707 val == 0.6667

Epoch 1/5000
82/82 [==============================] - 182s 2s/step - loss: 1.0641 - acc: 0.4451 - val_loss: 1.0373 - val_acc: 0.5000
Epoch 2/5000
82/82 [==============================] - 143s 2s/step - loss: 1.0616 - acc: 0.4360 - val_loss: 1.0349 - val_acc: 0.5833
Epoch 3/5000
82/82 [==============================] - 143s 2s/step - loss: 1.0280 - acc: 0.5091 - val_loss: 1.0099 - val_acc: 0.6389
Epoch 4/5000
82/82 [==============================] - 142s 2s/step - loss: 0.9995 - acc: 0.5640 - val_loss: 0.9976 - val_acc: 0.6667
Epoch 5/5000
82/82 [==============================] - 142s 2s/step - loss: 0.9530 - acc: 0.6067 - val_loss: 0.9819 - val_acc: 0.6111
Epoch 6/5000
82/82 [==============================] - 143s 2s/step - loss: 0.9314 - acc: 0.5945 - val_loss: 0.9731 - val_acc: 0.5833
Epoch 7/5000
82/82 [==============================] - 142s 2s/step - loss: 0.9351 - acc: 0.5915 - val_loss: 0.9754 - val_acc: 0.5000
Epoch 8/5000
82/82 [==============================] - 143s 2s/step - 

Process ForkPoolWorker-341:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
Process ForkPoolWorker-340:
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File 

KeyboardInterrupt: ignored

In [0]:
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [0]:
import gc
gc.collect()

146

In [0]:
##CNN + FC 1024 x 1 + GAP + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 1024))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9268 val == 0.6944

Epoch 1/5000
82/82 [==============================] - 261s 3s/step - loss: 1.0977 - acc: 0.5732 - val_loss: 0.8857 - val_acc: 0.5278
Epoch 2/5000
82/82 [==============================] - 216s 3s/step - loss: 0.7598 - acc: 0.7409 - val_loss: 0.8859 - val_acc: 0.5833
Epoch 3/5000
82/82 [==============================] - 216s 3s/step - loss: 0.6152 - acc: 0.7927 - val_loss: 1.4047 - val_acc: 0.6111
Epoch 4/5000
82/82 [==============================] - 216s 3s/step - loss: 0.5249 - acc: 0.8049 - val_loss: 0.9757 - val_acc: 0.4444
Epoch 5/5000
82/82 [==============================] - 216s 3s/step - loss: 0.4999 - acc: 0.8476 - val_loss: 0.8661 - val_acc: 0.6111
Epoch 6/5000
82/82 [==============================] - 216s 3s/step - loss: 0.4430 - acc: 0.8628 - val_loss: 0.9610 - val_acc: 0.5278
Epoch 7/5000
82/82 [==============================] - 217s 3s/step - loss: 0.5067 - acc: 0.8445 - val_loss: 0.9558 - val_acc: 0.5000
Epoch 8/5000
82/82 [==============================] - 216s 3s/step - 

Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/

KeyboardInterrupt: ignored

AveragePooling + Flatten





In [0]:
##CNN + FC 512 x 1 + AP + Flatten + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 512))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9329 val == 0.6944





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5000
82/82 [==============================] - 96s 1s/step - loss: 1.0871 - acc: 0.5976 - val_loss: 0.8592 - val_acc: 0.5000
Epoch 2/5000
82/82 [==============================] - 63s 771ms/step - loss: 0.8000 - acc: 0.7378 - val_loss: 0.7675 - val_acc: 0.5556
Epoch 3/5000
82/82 [==============================] - 63s 772ms/step - loss: 0.5395 - acc: 0.8140 - val_loss: 0.8170 - val_acc: 0.5000
Epoch 4/5000
82/82 [==============================] - 63s 770ms/step - loss: 0.4834 - acc: 0.8476 - val_loss: 0.8732 - val_acc: 0.4722
Epoch 5/5000
82/82 [==============================] - 63s 771ms/step - loss: 0.4061 - acc: 0.8537 - val_loss: 0.7469 - val_acc: 0.5556
Epoch 6/5000
82/82 [==============================] - 64s 776ms/step - loss: 0.4878 - acc: 0.8415 - val_loss: 0.8889 - val_acc: 0.5833
Epoch 7/5000
82/82 [==============================] - 63s 769ms/step - loss: 0.4689 - acc: 0.

Process ForkPoolWorker-46:
Process ForkPoolWorker-45:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 256 x 1 + AP + FLatten + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 256))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9451 val == 0.6944

Epoch 1/5000
81/82 [============================>.] - ETA: 0s - loss: 0.9617 - acc: 0.6080

82/82 [==============================] - 87s 1s/step - loss: 0.9566 - acc: 0.6098 - val_loss: 0.9422 - val_acc: 0.5833
Epoch 2/5000
82/82 [==============================] - 63s 765ms/step - loss: 0.7008 - acc: 0.7409 - val_loss: 0.8438 - val_acc: 0.6389
Epoch 3/5000
82/82 [==============================] - 62s 759ms/step - loss: 0.6597 - acc: 0.7683 - val_loss: 0.8509 - val_acc: 0.5278
Epoch 4/5000
82/82 [==============================] - 62s 756ms/step - loss: 0.4983 - acc: 0.8110 - val_loss: 0.8119 - val_acc: 0.5278
Epoch 5/5000
82/82 [==============================] - 62s 762ms/step - loss: 0.5501 - acc: 0.8079 - val_loss: 0.8898 - val_acc: 0.5556
Epoch 6/5000
82/82 [==============================] - 63s 765ms/step - loss: 0.3831 - acc: 0.8689 - val_loss: 0.8871 - val_acc: 0.5556
Epoch 7/5000
82/82 [==============================] - 62s 756ms/step - loss: 0.4532 - acc: 0.8445 - val_loss: 0.86

Process ForkPoolWorker-93:
Process ForkPoolWorker-94:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessin

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 128 x 1 + AP + FLatten + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 128))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9177 val == 0.6667

Epoch 1/5000
81/82 [============================>.] - ETA: 0s - loss: 0.9398 - acc: 0.6019

82/82 [==============================] - 90s 1s/step - loss: 0.9324 - acc: 0.6037 - val_loss: 0.8281 - val_acc: 0.5000
Epoch 2/5000
82/82 [==============================] - 62s 752ms/step - loss: 0.6649 - acc: 0.7713 - val_loss: 0.7698 - val_acc: 0.4444
Epoch 3/5000
82/82 [==============================] - 62s 756ms/step - loss: 0.6659 - acc: 0.7409 - val_loss: 0.7461 - val_acc: 0.5833
Epoch 4/5000
82/82 [==============================] - 62s 753ms/step - loss: 0.6884 - acc: 0.7530 - val_loss: 0.8095 - val_acc: 0.5278
Epoch 5/5000
82/82 [==============================] - 62s 754ms/step - loss: 0.4923 - acc: 0.8293 - val_loss: 0.8064 - val_acc: 0.5556
Epoch 6/5000
82/82 [==============================] - 62s 755ms/step - loss: 0.4520 - acc: 0.8537 - val_loss: 0.8495 - val_acc: 0.6667
Epoch 7/5000
82/82 [==============================] - 62s 755ms/step - loss: 0.4137 - acc: 0.8567 - val_loss: 0.87

Process ForkPoolWorker-141:
Process ForkPoolWorker-142:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File 

KeyboardInterrupt: ignored

In [0]:
##CNN + FC 64 x 1 + AP + FLatten + Leakyrelu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 64))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9451 val == 0.6944

Epoch 1/5000
82/82 [==============================] - 90s 1s/step - loss: 1.0899 - acc: 0.5274 - val_loss: 1.1593 - val_acc: 0.3056
Epoch 2/5000
82/82 [==============================] - 61s 750ms/step - loss: 0.8476 - acc: 0.6463 - val_loss: 1.3108 - val_acc: 0.2500
Epoch 3/5000
82/82 [==============================] - 61s 750ms/step - loss: 0.6469 - acc: 0.7470 - val_loss: 1.0193 - val_acc: 0.3611
Epoch 4/5000
82/82 [==============================] - 61s 746ms/step - loss: 0.6878 - acc: 0.7195 - val_loss: 0.9489 - val_acc: 0.4722
Epoch 5/5000
82/82 [==============================] - 62s 753ms/step - loss: 0.5495 - acc: 0.7774 - val_loss: 0.8658 - val_acc: 0.4722
Epoch 6/5000
82/82 [==============================] - 62s 753ms/step - loss: 0.5317 - acc: 0.8049 - val_loss: 0.8921 - val_acc: 0.3889
Epoch 7/5000
82/82 [==============================] - 61s 748ms/step - loss: 0.4663 - acc: 0.8415 - val_loss: 0.8840 - val_acc: 0.3889
Epoch 8/5000
82/82 [==============================] - 62s 

Process ForkPoolWorker-174:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [0]:
##CNN + FC 256 x 1 + AP + FLatten + relu
model = Sequential()
model.add(efficient_net)
model.add(AveragePooling2D(pool_size=(4, 4)))
model.add(Flatten(name="flatten"))
model.add(Dropout(0.5))
#model.add(Dense(units = 8192))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
#model.add(Dropout(0.4))
model.add(Dense(units = 256, activation ='relu'))
#model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dropout(0.4))
model.add(Dense(units = 3, activation ='softmax'))
opt = adam(lr=0.00001, amsgrad=True)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy']) #sparse_categorical_crossentropy for sequence
saver = keras.callbacks.ModelCheckpoint('/content/cnnfcIII.h5', save_best_only=True, monitor='val_acc', mode='max', save_weights_only=False, period=3)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, callbacks=[saver], steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, epochs=5000, validation_data = validation_generator, use_multiprocessing=True)
## train max == 0.9482 val == 0.6667

Epoch 1/5000
82/82 [==============================] - 95s 1s/step - loss: 0.9473 - acc: 0.6341 - val_loss: 0.8996 - val_acc: 0.4722
Epoch 2/5000
82/82 [==============================] - 63s 764ms/step - loss: 0.7990 - acc: 0.7104 - val_loss: 1.0548 - val_acc: 0.3333
Epoch 3/5000
82/82 [==============================] - 63s 764ms/step - loss: 0.7343 - acc: 0.7530 - val_loss: 0.9078 - val_acc: 0.4167
Epoch 4/5000
82/82 [==============================] - 62s 759ms/step - loss: 0.6455 - acc: 0.7470 - val_loss: 0.8774 - val_acc: 0.4722
Epoch 5/5000
82/82 [==============================] - 63s 768ms/step - loss: 0.5390 - acc: 0.8018 - val_loss: 0.8235 - val_acc: 0.5556
Epoch 6/5000
82/82 [==============================] - 63s 763ms/step - loss: 0.5000 - acc: 0.8110 - val_loss: 0.8600 - val_acc: 0.4722
Epoch 7/5000
82/82 [==============================] - 62s 761ms/step - loss: 0.4488 - acc: 0.8171 - val_loss: 0.8041 - val_acc: 0.5556
Epoch 8/5000
82/82 [==============================] - 63s 